In [1]:
import csv
import pandas as pd
import networkx as nx

In [2]:
data_file = '/global/D1/projects/umod/dipp/Misinformation_Dataset/Result/location_with_value/Location_with_only_country_code_cat_7_only_3.csv'
population_data_file = '/home/dipp/Github/Master-Thesis-dipp/Misinformation_Dataset_Analysis/Country_Data_Extraction/Result/Country_Data_cat_7_only_3.csv'

In [3]:
## Includes self Loops

country_data = {}
with open(data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    count = 0
    for row in reader:
        if(row['i'] != 'None' and row['j'] != 'None'):
            
            souce_loc = row['i'].replace("'","")
            destination_loc = row['j'].replace("'","")
            #country_data[count] = [row['i'], row['j'], float(row['count'])]
            #if(float(row['count']) > 999):
            country_data[count] = [souce_loc, destination_loc, float(row['count'])]
            count+=1

In [4]:
df = pd.DataFrame.from_dict(country_data, orient='index', columns=['Source','Destination','Number of Tweets'])

In [5]:
df

,Source,Destination,Number of Tweets
0,US,US,2638.0
1,GB,GB,387.0
2,CA,CA,107.0
3,IN,IN,116.0
4,JP,JP,5.0
...,...,...,...
225,AE,AE,1.0
226,NL,GB,1.0
227,PK,US,1.0
228,CN,GB,1.0


## Creating Graph using Networkx

In [6]:
G = nx.from_pandas_edgelist(df, source='Source', target='Destination', edge_attr='Number of Tweets', create_using=nx.DiGraph())

In [7]:
G.degree(weight='Number of Tweets')['US']
G.in_degree(weight='Number of Tweets')['US']
G.out_degree(weight='Number of Tweets')['US']

2915.0

In [8]:
country = []
deg = G.degree(weight='Number of Tweets')
deg_in = G.in_degree(weight='Number of Tweets')
deg_out = G.out_degree(weight='Number of Tweets')

In [9]:
df['All Degree of Source'] = ''
df['All Degree of Destination'] = ''
df['Vi - Out Degree of Source'] = ''
df['Vj - In Degree of Destination'] = ''
# df['Percentage of Incoming'] = ''
# df['Total No. of Tweets/Population'] = ''
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination
0,US,US,2638.0,,,,
1,GB,GB,387.0,,,,
2,CA,CA,107.0,,,,
3,IN,IN,116.0,,,,
4,JP,JP,5.0,,,,
...,...,...,...,...,...,...,...
225,AE,AE,1.0,,,,
226,NL,GB,1.0,,,,
227,PK,US,1.0,,,,
228,CN,GB,1.0,,,,


In [10]:
for index_df, row in df.iterrows():
    i = row['Source']
    j = row['Destination']

    for country_temp, deg_all in deg:
        if(country_temp == i):
            df.at[index_df, 'All Degree of Source'] = deg_all
        if(country_temp == j):
            df.at[index_df, 'All Degree of Destination'] = deg_all
    
    # Vj is in degree of destination

    for country_temp_in, deg_j in deg_in:
        if(country_temp_in == j):
            df.at[index_df, 'Vj - In Degree of Destination'] = deg_j
        # else:
        #     df.at[index_df, 'Vj - In Degree of Destination'] = 0
        

    # Vi is out degree of source

    for country_temp_out, deg_i in deg_out:
        if(country_temp_out == i):
            df.at[index_df, 'Vi - Out Degree of Source'] = deg_i
        # else:
        #     df.at[index_df, 'Vi - Out Degree of Source'] = 0    


In [11]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination
0,US,US,2638.0,5836.0,5836.0,2915.0,2921.0
1,GB,GB,387.0,1042.0,1042.0,527.0,515.0
2,CA,CA,107.0,338.0,338.0,179.0,159.0
3,IN,IN,116.0,290.0,290.0,156.0,134.0
4,JP,JP,5.0,15.0,15.0,6.0,9.0
...,...,...,...,...,...,...,...
225,AE,AE,1.0,2.0,2.0,1.0,1.0
226,NL,GB,1.0,23.0,1042.0,6.0,515.0
227,PK,US,1.0,7.0,5836.0,3.0,2921.0
228,CN,GB,1.0,18.0,1042.0,15.0,515.0


In [12]:
df['Percentage of Incoming at Destination (Total Tweet / Vj)'] = ''
df['Percentage of Outgoing From Source (Total Tweet / Vi)'] = ''
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi)
0,US,US,2638.0,5836.0,5836.0,2915.0,2921.0,,
1,GB,GB,387.0,1042.0,1042.0,527.0,515.0,,
2,CA,CA,107.0,338.0,338.0,179.0,159.0,,
3,IN,IN,116.0,290.0,290.0,156.0,134.0,,
4,JP,JP,5.0,15.0,15.0,6.0,9.0,,
...,...,...,...,...,...,...,...,...,...
225,AE,AE,1.0,2.0,2.0,1.0,1.0,,
226,NL,GB,1.0,23.0,1042.0,6.0,515.0,,
227,PK,US,1.0,7.0,5836.0,3.0,2921.0,,
228,CN,GB,1.0,18.0,1042.0,15.0,515.0,,


In [13]:
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']
    vi = row['Vi - Out Degree of Source']
    vj = row['Vj - In Degree of Destination']

    per_in = float(No_of_Tweets / vj) * 100
    per_out = float(No_of_Tweets / vi) * 100
    df.at[index_df, 'Percentage of Incoming at Destination (Total Tweet / Vj)'] = per_in
    df.at[index_df, 'Percentage of Outgoing From Source (Total Tweet / Vi)'] = per_out

    


In [14]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi)
0,US,US,2638.0,5836.0,5836.0,2915.0,2921.0,90.311537,90.497427
1,GB,GB,387.0,1042.0,1042.0,527.0,515.0,75.145631,73.434535
2,CA,CA,107.0,338.0,338.0,179.0,159.0,67.295597,59.776536
3,IN,IN,116.0,290.0,290.0,156.0,134.0,86.567164,74.358974
4,JP,JP,5.0,15.0,15.0,6.0,9.0,55.555556,83.333333
...,...,...,...,...,...,...,...,...,...
225,AE,AE,1.0,2.0,2.0,1.0,1.0,100.0,100.0
226,NL,GB,1.0,23.0,1042.0,6.0,515.0,0.194175,16.666667
227,PK,US,1.0,7.0,5836.0,3.0,2921.0,0.034235,33.333333
228,CN,GB,1.0,18.0,1042.0,15.0,515.0,0.194175,6.666667


## Extensions

### Tweets per population and 2 versions of Cij 

In [15]:
population_data = {}

with open(population_data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    
    for row in reader:
        # print(row['Country Code'])
        # print(row['Population'])
        if(row['Population']!=''):
            population_data[row['Country Code']] = int(row['Population'])


In [16]:
population_data['AI'] = 15753
population_data['AQ'] = 3000
population_data['AX'] = 30129
population_data['BQ'] = 27726
population_data['CX'] = 1692
population_data['EH'] = 612000
population_data['FK'] = 3662
population_data['GF'] = 301099
population_data['GG'] = 63950
population_data['GS'] = 1
population_data['IO'] = 3000
population_data['JE'] = 103267
population_data['PN'] = 47
population_data['RE'] = 873102
population_data['SH'] = 4439
population_data['TF'] = 300
population_data['TW'] = 23894394
population_data['UM'] = 300
population_data['VA'] = 453
population_data['YT'] = 310022

In [17]:
df['Tweets Per Population'] = ''
df['Cij (All Degrees)'] = ''
df['Cij'] = ''

In [18]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,US,US,2638.0,5836.0,5836.0,2915.0,2921.0,90.311537,90.497427,,,
1,GB,GB,387.0,1042.0,1042.0,527.0,515.0,75.145631,73.434535,,,
2,CA,CA,107.0,338.0,338.0,179.0,159.0,67.295597,59.776536,,,
3,IN,IN,116.0,290.0,290.0,156.0,134.0,86.567164,74.358974,,,
4,JP,JP,5.0,15.0,15.0,6.0,9.0,55.555556,83.333333,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
225,AE,AE,1.0,2.0,2.0,1.0,1.0,100.0,100.0,,,
226,NL,GB,1.0,23.0,1042.0,6.0,515.0,0.194175,16.666667,,,
227,PK,US,1.0,7.0,5836.0,3.0,2921.0,0.034235,33.333333,,,
228,CN,GB,1.0,18.0,1042.0,15.0,515.0,0.194175,6.666667,,,


In [19]:
import pypopulation

In [20]:
temp = 0
None_Type_Countries = []
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']


    # source_poplation = population_data.get(source)
    # dest_population = population_data.get(destination)


    source_poplation = pypopulation.get_population(source)
    dest_population = pypopulation.get_population(destination)

    if(source_poplation == None):
        source_poplation = population_data.get(source)
            

    if(dest_population == None):
        dest_population = population_data.get(destination)
            

    percentage = No_of_Tweets / (source_poplation + dest_population)

    df.at[index_df, 'Tweets Per Population'] = percentage

    

In [21]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,US,US,2638.0,5836.0,5836.0,2915.0,2921.0,90.311537,90.497427,0.000004,,
1,GB,GB,387.0,1042.0,1042.0,527.0,515.0,75.145631,73.434535,0.000003,,
2,CA,CA,107.0,338.0,338.0,179.0,159.0,67.295597,59.776536,0.000001,,
3,IN,IN,116.0,290.0,290.0,156.0,134.0,86.567164,74.358974,0.0,,
4,JP,JP,5.0,15.0,15.0,6.0,9.0,55.555556,83.333333,0.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...
225,AE,AE,1.0,2.0,2.0,1.0,1.0,100.0,100.0,0.0,,
226,NL,GB,1.0,23.0,1042.0,6.0,515.0,0.194175,16.666667,0.0,,
227,PK,US,1.0,7.0,5836.0,3.0,2921.0,0.034235,33.333333,0.0,,
228,CN,GB,1.0,18.0,1042.0,15.0,515.0,0.194175,6.666667,0.0,,


In [22]:
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']
    all_degree_source = row['All Degree of Source']
    all_degree_dest = row['All Degree of Destination']
    vi = row['Vi - Out Degree of Source']
    vj = row['Vj - In Degree of Destination']


    cij_all = No_of_Tweets / (all_degree_source + all_degree_dest)
    cij = No_of_Tweets / (vi + vj)

    df.at[index_df, 'Cij (All Degrees)'] = cij_all
    df.at[index_df, 'Cij'] = cij
    

In [23]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,US,US,2638.0,5836.0,5836.0,2915.0,2921.0,90.311537,90.497427,0.000004,0.226011,0.452022
1,GB,GB,387.0,1042.0,1042.0,527.0,515.0,75.145631,73.434535,0.000003,0.185701,0.371401
2,CA,CA,107.0,338.0,338.0,179.0,159.0,67.295597,59.776536,0.000001,0.158284,0.316568
3,IN,IN,116.0,290.0,290.0,156.0,134.0,86.567164,74.358974,0.0,0.2,0.4
4,JP,JP,5.0,15.0,15.0,6.0,9.0,55.555556,83.333333,0.0,0.166667,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...
225,AE,AE,1.0,2.0,2.0,1.0,1.0,100.0,100.0,0.0,0.25,0.5
226,NL,GB,1.0,23.0,1042.0,6.0,515.0,0.194175,16.666667,0.0,0.000939,0.001919
227,PK,US,1.0,7.0,5836.0,3.0,2921.0,0.034235,33.333333,0.0,0.000171,0.000342
228,CN,GB,1.0,18.0,1042.0,15.0,515.0,0.194175,6.666667,0.0,0.000943,0.001887


In [24]:
df.to_csv('/home/dipp/Github/Master-Thesis-dipp/Misinformation_Dataset_Analysis/Network_Analysis/Result/global_network_cat_7_only_3.csv', index=False)